In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

* I have included all the required libraries in above cell.
* Pandas and Numpy are used for data importing and data extracting to feed the for other libraries 
* Nltk is used for removing pre-processing of corpus
* Word2Vec is also a preprocessing technique which is a neural network and helps in retaining semantic as well as syntatic information.

In [2]:
df=pd.read_csv("ecommerceDataset.csv",sep=',',names=["category","description"])

* I have imported the file using pandas 

In [3]:
df

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50420,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50421,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50422,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50423,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [4]:
df.isnull().sum()

category       0
description    1
dtype: int64

In [5]:
df=df.dropna()
df=df.reset_index(drop=True)

* In the above cell I have removed the one row which contains NAN value .

In [7]:
corpus=[]
for i in range(0,len(df)):
    review=re.sub('[^a-zA-Z]',' ',df['description'][i])
    review=review.lower()
    review=review.split()
    review=[words for words in review if not words in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

* In the above I am doing all the preprocessing of the data. I am removing all the stopwords such as I ,We ,They etc...
* I am also removing any character except for Alphabets because alphabets form words and words are only required for the           training an NLP model.
* I am also lower casing all the words because I dont lower case "Apple" and "apple" will be considered two diffferent words in   my corpus , which increases the vulnerability of overfitting.

In [8]:
corpus1=[]
w_n=WordNetLemmatizer()
for i in range(0,len(corpus)):
    review=corpus[i]
    review=[w_n.lemmatize(words) for words in review]
    review=''.join(review)
    corpus1.append(review)


* In the above code I have lemmatized my corpus which is a way of preprocessing technique

In [9]:
Bigger_list=[]
for i in corpus1:
     sub=list(i.split(' '))
     Bigger_list.append(sub)


* I have converted my "corpus1" into list of lists that is "Bigger_list". I have done this because word2Vec only accpets data     which is in form of list whithin a list.

In [11]:
Y=df.iloc[:,0].values
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

* I have converted my target variable into NLP model presentable  target varibale by using LabelEncoder and to_categorical. As     the target variable contains characters and words they need to converted into model understandble form as at the end of the     day machine learning is all about numbers.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (Bigger_list,dummy_y, test_size=0.2)

* I have splitted the corpus into Training and testing data. It is standard procedure followed by people to ensure they is no     overfitting on all data and testing data is used for validation of the predicted data.

In [13]:
w2v_model = Word2Vec(X_train,vector_size=500,min_count=2)


* I have implemented the Word2Vec neural network . The vector_size is of 500 and min_count of each word in vector is 2

In [14]:
w2v_model.wv.index_to_key


['x',
 'size',
 'book',
 'set',
 'use',
 'one',
 'easy',
 'quality',
 'high',
 'black',
 'also',
 'product',
 'made',
 'home',
 'design',
 'author',
 'cm',
 'color',
 'mm',
 'new',
 'time',
 'power',
 'cotton',
 'best',
 'usb',
 'perfect',
 'light',
 'comes',
 'make',
 'features',
 'used',
 'material',
 'women',
 'water',
 'long',
 'designed',
 'steel',
 'life',
 'free',
 'inch',
 'white',
 'style',
 'look',
 'like',
 'world',
 'table',
 'books',
 'great',
 'well',
 'makes',
 'men',
 'soft',
 'years',
 'camera',
 'top',
 'india',
 'colour',
 'range',
 'fit',
 'cable',
 'bluetooth',
 'audio',
 'fabric',
 'name',
 'stainless',
 'room',
 'kitchen',
 'wear',
 'v',
 'wireless',
 'work',
 'plastic',
 'get',
 'products',
 'even',
 'easily',
 'w',
 'wall',
 'without',
 'experience',
 'keep',
 'g',
 'pack',
 'air',
 'brand',
 'hand',
 'full',
 'good',
 'way',
 'system',
 'blue',
 'every',
 'length',
 'storage',
 'led',
 'comfortable',
 'clean',
 'review',
 'comfort',
 'two',
 'different',
 'dev

In [16]:
emb_df = (
    pd.DataFrame(
        [w2v_model.wv.get_vector(str(n)) for n in w2v_model.wv.key_to_index],
        index = w2v_model.wv.key_to_index
    )
)
emb_df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
x,0.200739,-0.195618,0.519429,1.080256,-0.750230,0.132412,-0.593700,0.956430,-0.607862,-0.126475,...,0.607304,-0.815473,0.263435,-0.427634,-0.031694,-0.624634,1.111764,0.484459,0.990387,0.528301
size,-0.825594,0.356079,0.573316,0.478548,-1.197532,-0.142884,0.034866,0.484288,-1.516412,0.059512,...,-1.144145,0.717527,0.102362,0.458191,1.506234,-0.885418,-0.990446,0.568659,1.034311,0.647028
book,-0.026073,-0.401894,1.868490,-1.545737,0.376533,-0.007971,-0.979492,-0.049779,-0.036002,-0.778212,...,0.348064,-0.347594,0.265368,0.615477,0.715497,-0.269789,-0.492135,-0.718247,-1.668540,-0.847314
set,0.511687,-0.163344,-0.047670,0.300243,-0.574466,0.678101,-0.460112,-0.059032,-0.644324,1.047236,...,-1.339235,-0.018179,-1.309904,1.475975,-1.224585,-0.983603,-0.242972,1.936243,0.770754,-0.009123
use,0.347441,0.642710,-0.139679,1.814078,-0.101757,0.459244,0.097853,0.742256,0.177960,0.823247,...,-0.055774,-1.155108,0.468200,0.802216,-0.532556,-1.618827,0.557069,0.000838,1.554201,-1.956362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
satva,0.006704,0.004658,0.008026,0.007224,-0.005851,-0.004528,-0.004777,0.011995,0.009675,0.000457,...,0.007588,0.004485,-0.001247,-0.000720,0.002118,-0.000915,0.003126,-0.012754,-0.001726,-0.009291
excelmost,0.012756,0.015102,0.006346,0.014200,0.013223,-0.000036,-0.003541,0.026129,0.005805,0.001942,...,0.016410,-0.001464,0.012047,-0.010643,0.003176,0.015003,0.009187,-0.009196,-0.013614,-0.001073
screenshots,0.011299,0.007594,0.020019,0.002228,0.014310,-0.001884,-0.001195,0.018966,0.003461,-0.005607,...,0.004951,-0.008279,0.027193,-0.010732,0.008167,0.006825,0.011112,-0.020901,-0.006072,-0.010377
dataestimating,0.011320,0.018989,0.012182,0.015734,-0.007901,-0.011619,0.018604,0.036855,0.010602,0.001165,...,0.007703,-0.002834,0.036071,0.004418,0.010796,0.012752,-0.003572,-0.021825,-0.031035,-0.013949


* The above data frame shows us the similarity of each word with the corpus obtained from word2Vec (the 500 words) . 
* The closer the they are the similar they are .

In [19]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

C:\Users\Mohd Arafat\AppData\Local\Temp\ipykernel_17280\3620852164.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


(40339,) (10085,)


C:\Users\Mohd Arafat\AppData\Local\Temp\ipykernel_17280\3620852164.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [18]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(500, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(500, dtype=float))

* The above are the pre processing of Word2vec to ensure that all the vectors have same size .

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train)

* I have used RandForestClassifier for classifing my data. I have choosen RandomForestClassifier randomly . There is no specific   reason for it.

In [21]:
y_pred = rf_model.predict(X_test_vect_avg)


* I have predicted the model work by using my "Testing Data".
* I have used metrics like precision , accuracy and recall_score to show how my model works .
* As I have used in my below cell all the performance metrics are pretty good.

In [22]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
accuracy=accuracy_score(y_test, y_pred)
print('''Precision: {} 
Recall: {} 
Accuracy: {}'''.format(
    precision, recall,accuracy ))

Precision: 0.982628777989758 
Recall: 0.9703520079325731 
Accuracy: 0.9703520079325731
